In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
from datetime import timedelta

In [1]:
def read_out_tr(dir, file):
    col = ["events", "time", "from_node", "to_node", "pkt_type", "pkt_size", "flags", "fid", "src_addr", "dst_addr", "seq_num", "pkt_id"]
    df = pd.read_csv(dir+file, sep = ' ', header = None)
    df.columns = col
    return df

# Monitoring queue length by Mengqi
def get_src_packets(df, switch):
    # get all the tcp pkts (Modified by Peixuan)
    df = df[["time","events", "pkt_type", "from_node", "to_node"]]
    df = df.loc[df["events"].isin(["+", "-", "d"])]
    df = df.loc[df["pkt_type"].isin(["tcp"])]
    df = df.loc[df["from_node"].isin([switch])]
    return df

def sort_by_time(df):    
    df = df.sort_values(by=["time", "events"])
    return df

def plot(df):
    plt.figure(figsize=(18,5))
    #plt.plot(x,y,"b--",linewidth=1)   #在当前绘图对象绘图（X轴，Y轴，蓝色虚线，线宽度）
    plt.xlabel("Time")
    plt.ylabel("Queue Length")
    #plt.xscale('log')
    plt.title("Real Time Queue Length")    
    #line0 = plt.plot(df["time"][200:500], df["queue_len"][200:500], color='red', markersize = 12)
    line0 = plt.plot(df["time"], df["queue_len"])
    line1 = plt.plot(df["time"], df["queue_drop"])
    plt.legend(loc='best')
    plt.show()

def cal_queue_len(df_out_tr, switch):
    df_queue = get_src_packets(df_out_tr, switch)
    df_queue = sort_by_time(df_queue)
    pktCount = 0
    queue_len = []
    pktDrop = 0
    queue_drop = []
    for index, row in df_queue.iterrows():
        if row["events"] == 'd': 
            pktDrop +=1
            pktCount -= 1
        if row["events"] == '-':
            pktCount -= 1
        if row["events"] == '+':
            pktCount += 1
        queue_len.append(pktCount)
        queue_drop.append(pktDrop)
    df_queue["queue_len"] = queue_len
    df_queue["queue_drop"] = queue_drop
    plot(df_queue)
    
    #return df_sort
def plot_flow_path_queue_length(df, fid):
    df_out = df.loc[df["pkt_type"].isin(["tcp"])]
    df_out = df_out.loc[df_out["fid"].isin([fid])]
    print("from", df_out.from_node.unique())
    print("to", df_out.to_node.unique())
    for switch in df_out.from_node.unique():
        cal_queue_len(df, switch)